In [22]:
import pandas as pd
import random
from ai.models.similarity_model import ImageSimilarityNetwork
from keras.models import Model
from sklearn.metrics.pairwise import cosine_similarity
from commons.image_utils import scale_image
import numpy as np
from PIL import Image
from keras.preprocessing import image

from commons.config import MVC_GENERATED_TRIPLETS_CSV, MVC_IMAGES_FOLDER, DEFAULT_IMAGE_SIZE

In [3]:
data_set = pd.read_csv(MVC_GENERATED_TRIPLETS_CSV)

In [6]:
trained_model = ImageSimilarityNetwork()
trained_model = trained_model.get_trained_model('/run/media/twoaday/data-storag/data-sets/multi-view-clothing/models/xception_similarity.01-0.00.hdf5')

Instructions for updating:
Colocations handled automatically by placer.


2019-06-21 10:42:32,989 : WARNING : From /usr/lib/python3.6/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


2019-06-21 10:42:34,089 : WARNING : From /usr/lib/python3.6/site-packages/tensorflow/python/training/moving_averages.py:210: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [23]:
def load_image(img_path):
    f = open(img_path, 'rb')
    f = Image.open(f)
    f = scale_image(f, [DEFAULT_IMAGE_SIZE, DEFAULT_IMAGE_SIZE])
    f = np.asarray(f)
    img_data = image.img_to_array(f)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = img_data.astype('float32') / 255.
    img_data = np.clip(img_data, 0., 1.)
    img_data = img_data[0]
    return img_data

In [45]:
sample = ['{0}{1}'.format(MVC_IMAGES_FOLDER, img) for img in data_set.sample(1).values.tolist()[0]]
vectors = [load_image(im_path) for im_path in sample]
vectors = trained_model.predict([vectors])

pos_dist = cosine_similarity([vectors[0]], [vectors[1]])
neg_dist = cosine_similarity([vectors[0]], [vectors[2]])

img = Image.open(sample[0])
img = img.resize((256, 256))
display(img)
print('Anchor image')

img = Image.open(sample[1])
img = img.resize((256, 256))
display(img)
print('Positive image, distance: {0}'.format(pos_dist))

img = Image.open(sample[2])
img = img.resize((256, 256))
display(img)
print('Negative image, distance: {0}'.format(neg_dist))

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').